<a href="https://colab.research.google.com/github/1901010019/1901010019.github.io/blob/master/Introduction_to_NLP_in_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# checking if we are using GPU
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-d2655200-f459-e75d-0604-aedb9ec71d71)


In [ ]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2021-08-06 15:39:16--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py.1’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2021-08-06 15:39:16 (71.9 MB/s) - ‘helper_functions.py.1’ saved [10246/10246]



In [ ]:
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

## GETTING THE DATASET

we will use kaggle dataset for our this experimentation.
https://www.kaggle.com/c/nlp-getting-started/data

In [ ]:
# getting a text dataset 
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

--2021-08-06 15:39:18--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.101.128, 142.250.141.128, 142.251.2.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.101.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip.1’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.004s  

2021-08-06 15:39:18 (151 MB/s) - ‘nlp_getting_started.zip.1’ saved [607343/607343]



In [ ]:
unzip_data('nlp_getting_started.zip') # unzipped the dataset

## VISUALIZING OUR TEXT DATASET

In [ ]:
# reading dataset using pandas library
import pandas as pd
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [ ]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
# shuffling training dataset so that our model doesnot learn any unwanted patterns.
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [ ]:
# how many examples of each class?
train_df_shuffled.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [ ]:
# how many total samples
len(train_df), len(test_df)

(7613, 3263)

In [ ]:
# visualize the random training examples
import random
random_index = random.randint(0, len(train_df)-5) 
for row in train_df_shuffled[['text','target']][random_index : random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)" )
  print(f"text:\n{text}\n")
  print("-----\n")

Target: 1 (real disaster)
text:
I Pledge Allegiance To The P.O.P.E. And The Burning Buildings of Epic City. ??????

-----

Target: 0 (not real disaster)
text:
@charlieputh that song have a Cool beat like Nothing But Trouble

-----

Target: 0 (not real disaster)
text:
IS THE UPDATE RLY LIFE NOW IS IT IS It/Screams 
vibrates i cant handle

-----

Target: 0 (not real disaster)
text:
Thank you @FedEx for no longer shipping live microbes for the Department of Defense

http://t.co/zAHNEwJrI8

-----

Target: 0 (not real disaster)
text:
I want to smoke ??

-----



### split data into training and validation sets


In [ ]:
from sklearn.model_selection import train_test_split
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled['text'].to_numpy(),
                                                                            train_df_shuffled['target'].to_numpy(),
                                                                            test_size=0.1,
                                                                            random_state=42)

In [ ]:
len(train_sentences), len(val_sentences), len(train_labels), len(val_labels)

(6851, 762, 6851, 762)

In [ ]:
train_sentences

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       ...,
       'Near them on the sand half sunk a shattered visage lies... http://t.co/0kCCG1BT06',
       "kesabaran membuahkan hasil indah pada saat tepat! life isn't about waiting for the storm to pass it's about learning to dance in the rain.",
       "@ScottDPierce @billharris_tv @HarrisGle @Beezersun I'm forfeiting this years fantasy football pool out of fear I may win n get my ass kicked"],
      dtype=object)

In [ ]:
train_labels

array([0, 0, 1, ..., 1, 1, 0])

In [ ]:
val_sentences[0:10] # viewing initial 10 sentences

array(['DFR EP016 Monthly Meltdown - On Dnbheaven 2015.08.06 http://t.co/EjKRf8N8A8 #Drum and Bass #heavy #nasty http://t.co/SPHWE6wFI5',
       'FedEx no longer to transport bioterror germs in wake of anthrax lab mishaps http://t.co/qZQc8WWwcN via @usatoday',
       'Gunmen kill four in El Salvador bus attack: Suspected Salvadoran gang members killed four people and wounded s... http://t.co/CNtwB6ScZj',
       '@camilacabello97 Internally and externally screaming',
       'Radiation emergency #preparedness starts with knowing to: get inside stay inside and stay tuned http://t.co/RFFPqBAz2F via @CDCgov',
       'Investigators rule catastrophic structural failure resulted in 2014 Virg.. Related Articles: http://t.co/Cy1LFeNyV8',
       'How the West was burned: Thousands of wildfires ablaze in #California alone http://t.co/iCSjGZ9tE1 #climate #energy http://t.co/9FxmN0l0Bd',
       "Map: Typhoon Soudelor's predicted path as it approaches Taiwan; expected to make landfall over southern C

In [ ]:
val_labels[0:30] # viewing initial 30 labels

array([0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 0])

## CONVERTING TEXT TO NUMBERS

In [ ]:
import tensorflow
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [ ]:
# use the default text vectorization parameters
text_vectorizer = TextVectorization(max_tokens=None, # how many words in the vocabluary
                                    standardize='lower_and_strip_punctuation',
                                    split='whitespace',
                                    ngrams=None,
                                    output_mode='int',
                                    output_sequence_length=None,
                                    pad_to_max_tokens=True)

In [ ]:
# find the average number of tokens (words) in the training tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))
# so there are 15 words in every tweet on average

15

In [ ]:
# setup text vectorization variables
max_vocab_length = 10000 # maximum number of words to have in our dictionary
max_length = 15 # max length our sequence will be
text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode = 'int',
                                    output_sequence_length=max_length)

In [ ]:
# fit the text vectorizer to the training text
text_vectorizer.adapt(train_sentences)

In [ ]:
# create a sample sentence and tokenize it
sample_sentence = ['hello, this is kunal from pakistan']
text_vectorizer(sample_sentence) # here we can see we have padded with zero's as there are zeros in the last to complete it to 15

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[1400,   19,    9,    1,   20,  811,    0,    0,    0,    0,    0,
           0,    0,    0,    0]])>

In [ ]:
# choose a random sentence from our training data and tokenize it
random_sentence = random.choice(train_sentences)
print(f"original sentence:\n {random_sentence} \
     \n\n vectorized_version:\n")
text_vectorizer([random_sentence])

original sentence:
 Those that I have sworn to defend have proven themselves to be friends of the House Hailstorm.      

 vectorized_version:



<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 161,   16,    8,   24, 7853,    5, 3065,   24, 9516, 2656,    5,
          21,  819,    6,    2]])>

In [ ]:
# getting the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary() 
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]
print(f"number of words in vocabulary: {len(words_in_vocab)}")
print(f"5 most commom words: {top_5_words}")
print(f"5 least common words: {bottom_5_words}")

number of words in vocabulary: 10000
5 most commom words: ['', '[UNK]', 'the', 'a', 'in']
5 least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


In [ ]:
# words_in_vocab -> we can see the vocabulary this way

## CREATING AN EMBEDDING USING AN EMBEDDING LAYER

In [ ]:
from tensorflow.keras import layers
embedding = layers.Embedding(input_dim = max_vocab_length, output_dim = 128, input_length = max_length)

In [ ]:
# get a random sentence from our training data
random_sentence = random.choice(train_sentences)
print(f"original texrt:\n {random_sentence} \
       \n\n embedded version:\n "
      )
# embed the random sentence (turn it into dense vectors of fixed size)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

original texrt:
 Hellfire is surrounded by desires so be careful and donÛªt let your desires control you! #Afterlife        

 embedded version:
 


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.00879746,  0.00621387, -0.01830555, ...,  0.02625204,
          0.03845856, -0.03402692],
        [ 0.04096503,  0.0404273 ,  0.00225029, ...,  0.02850764,
          0.0412682 , -0.01777378],
        [ 0.00063924,  0.03514132,  0.03975764, ..., -0.02449412,
         -0.04623724, -0.04647881],
        ...,
        [-0.03694429,  0.04591988, -0.03658252, ..., -0.02619305,
          0.03823229,  0.03003428],
        [-0.00154668,  0.03742785, -0.01729655, ..., -0.00078506,
          0.00016145,  0.0225885 ],
        [-0.04237081,  0.0033442 , -0.01128031, ...,  0.0392071 ,
          0.0069814 ,  0.02264117]]], dtype=float32)>

In [ ]:
# check out a single token's embed
sample_embed[0][0], sample_embed[0][0].shape, random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([ 0.00879746,  0.00621387, -0.01830555,  0.04479036,  0.03027774,
        -0.02478739, -0.03306929,  0.03885527, -0.02847859, -0.02596858,
        -0.02603444,  0.04729625, -0.02172185, -0.04063044,  0.04486314,
         0.04478041, -0.03796153,  0.00596523,  0.02974595,  0.04885942,
        -0.00415882,  0.01400635,  0.04580193, -0.02681954,  0.04787714,
        -0.02223131,  0.04336068, -0.00268724,  0.0200294 , -0.03207554,
        -0.01109829,  0.02402462,  0.02373468,  0.00594934, -0.04533259,
        -0.01487494, -0.02131438, -0.0164215 ,  0.01414062, -0.02816062,
         0.01651276,  0.0261997 ,  0.0366511 ,  0.01052599, -0.00739105,
         0.04477075, -0.04909487, -0.02700547, -0.04251846,  0.00624407,
        -0.04205314, -0.03885403, -0.015652  ,  0.00533151, -0.02263205,
        -0.03891248, -0.02711318, -0.01073035,  0.04685471, -0.03221074,
         0.02119908, -0.00691849, -0.00235472, -0.00857655,  0.00042721,
  

### MODEL 0: NAIVE BAYES (BASELINE)

* we will use `tfidf` to convert our texts into numbers

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# create tokenization and modelling pipeline
model_0 = Pipeline(
    [
     ('tfidf', TfidfVectorizer()), # convert words to numbers using tfidf
     ('clf', MultinomialNB()) # model the text
    ]
)

# fit the pipeline to our training data
model_0.fit(train_sentences, train_labels)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [ ]:
# evaluate our baseline model
baseline_score = model_0.score(val_sentences, val_labels)
# default evaluatio matrix in scikit learn is accuracy

In [ ]:
print(f"our baseline model achieves an accuracy of: {baseline_score*100:.2f} %")

our baseline model achieves an accuracy of: 79.27 %


In [ ]:
# we will predict on our validation data now
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

creating an evaluation function for our model experiments. we will use the following metrics for evaluation.


1.   Accuracy
2.   Precision
3.   Recall
4.   F1-score



In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_score(y_true, y_pred):
  """
  calculates model accuracy, precision, recall, f1-score
  """
  # calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # calculate model precision, recall, f1-score using weighted score
  precision, recall, f1_score, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')
  model_results = {
      'accuracy': model_accuracy,
      'precision': precision,
      'recall': recall,
      'f1-score': f1_score
  }
  return model_results

In [ ]:
# get baseline results
baseline_results = calculate_score(y_true=val_labels,
                                     y_pred = baseline_preds,
                                     )
baseline_results

{'accuracy': 79.26509186351706,
 'f1-score': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

### MODEL 2: A SIMPLE DENSE MODEL

In [ ]:
# create a tensorboard callback
from helper_functions import create_tensorboard_callback

# creating a directory to save our tensorboard logs
save_dir = 'model_logs'

In [ ]:
# build model with functional API
import tensorflow as tf
from tensorflow.keras import layers
input = layers.Input(shape=(1,), dtype=tf.string) # inputs are one dimensionl strings
x = text_vectorizer(input) # turn the inputs into numbers
x = embedding(x) # create an embedding of the numberized inputs
x = layers.GlobalAveragePooling1D()(x) # condense the feature vector for each token to one vector
output = layers.Dense(1, activation='sigmoid')(x) # create the output layer
model_1 = tf.keras.Model(input, output, name='model_1_dense')

In [ ]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
global_average_pooling1d (Gl (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 1,280,129
Trainable params: 1,280,129
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# compile model_1
model_1.compile(loss = 'binary_crossentropy',
                optimizer = 'adam',
                metrics=['accuracy'])

In [ ]:
# fit the model
model_1_history = model_1.fit(train_sentences, train_labels, epochs=5, validation_data=(val_sentences, val_labels),
                              callbacks = [create_tensorboard_callback(dir_name=save_dir, experiment_name='model_1_dense')])

Saving TensorBoard log files to: model_logs/model_1_dense/20210806-153921
Epoch 1/5
215/215 [==============================] - 4s 16ms/step - loss: 0.6135 - accuracy: 0.6897 - val_loss: 0.5381 - val_accuracy: 0.7533
Epoch 2/5
215/215 [==============================] - 3s 13ms/step - loss: 0.4434 - accuracy: 0.8184 - val_loss: 0.4682 - val_accuracy: 0.7808
Epoch 3/5
215/215 [==============================] - 3s 14ms/step - loss: 0.3471 - accuracy: 0.8619 - val_loss: 0.4612 - val_accuracy: 0.7979
Epoch 4/5
215/215 [==============================] - 3s 14ms/step - loss: 0.2844 - accuracy: 0.8901 - val_loss: 0.4637 - val_accuracy: 0.7822
Epoch 5/5
215/215 [==============================] - 3s 14ms/step - loss: 0.2390 - accuracy: 0.9092 - val_loss: 0.4810 - val_accuracy: 0.7848


In [ ]:
# check the results
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 3ms/step - loss: 0.4810 - accuracy: 0.7848


[0.48104384541511536, 0.7847769260406494]

In [ ]:
# make some predictions and evaluate those
model_1_pred_prob = model_1.predict(val_sentences)
model_1_pred_prob.shape
model_1_pred_prob

array([[4.01041716e-01],
       [7.40684152e-01],
       [9.97584581e-01],
       [1.32033661e-01],
       [1.27762318e-01],
       [9.37868774e-01],
       [9.13600028e-01],
       [9.92623448e-01],
       [9.67914164e-01],
       [2.86270261e-01],
       [1.30230144e-01],
       [6.82497144e-01],
       [5.32644019e-02],
       [1.91140458e-01],
       [4.32200730e-03],
       [1.46771535e-01],
       [2.58098207e-02],
       [9.75843295e-02],
       [2.42753342e-01],
       [5.74996352e-01],
       [9.09612954e-01],
       [4.04059291e-02],
       [4.13487256e-01],
       [9.20515358e-02],
       [9.56371963e-01],
       [9.99079823e-01],
       [3.75315212e-02],
       [7.46415630e-02],
       [2.91814860e-02],
       [2.07112685e-01],
       [5.75213969e-01],
       [2.89756984e-01],
       [4.36146468e-01],
       [1.82937488e-01],
       [4.84587908e-01],
       [5.39458394e-02],
       [9.93951201e-01],
       [1.61340892e-01],
       [2.74551269e-02],
       [9.98447239e-01],


In [ ]:
# convert model prediction probabilities to label format
model_1_preds = tf.squeeze(tf.round(model_1_pred_prob))
model_1_preds

<tf.Tensor: shape=(762,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 0.,
       1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 0.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0.,
       0., 0., 0., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0.,
       0., 1., 1., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1.,
       0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0.,
       1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0.,
       1., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1., 0., 0.,
       1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0.,
       0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0

In [ ]:
# calculate our model_1 results
model_1_results = calculate_score(val_labels,model_1_preds)
model_1_results

{'accuracy': 78.4776902887139,
 'f1-score': 0.7818959205825942,
 'precision': 0.789165199286798,
 'recall': 0.7847769028871391}

### VISUALIZING LEARNED EMBEDDINGS

In [ ]:
# get vocabulary from our text vectorization layer
words_in_vocab = text_vectorizer.get_vocabulary() # we have previously also instantiated it, but for better visualization we are doing it again here.
len(words_in_vocab), words_in_vocab[:20] # twenty most frequent words

(10000,
 ['',
  '[UNK]',
  'the',
  'a',
  'in',
  'to',
  'of',
  'and',
  'i',
  'is',
  'for',
  'on',
  'you',
  'my',
  'with',
  'it',
  'that',
  'at',
  'by',
  'this'])

In [ ]:
max_vocab_length

10000

In [ ]:
# get the weight matrix of our embedding layer which has been learned over 5 epochs
embed_weights = model_1.get_layer('embedding').get_weights()[0]
embed_weights , embed_weights.shape # for every token in our vocab, it is represented by total of 128 vectors

(array([[ 0.04243163,  0.0016714 ,  0.01379535, ...,  0.03299421,
          0.06340357, -0.05303558],
        [-0.04468029,  0.02047528, -0.03755324, ...,  0.00159528,
          0.04219921, -0.01509366],
        [ 0.01335387,  0.02747238, -0.02408821, ...,  0.02386842,
          0.05091142, -0.02941865],
        ...,
        [ 0.03385148, -0.02602719,  0.04982534, ...,  0.03260219,
         -0.01196181, -0.0478712 ],
        [ 0.0844968 , -0.00144431, -0.00862418, ...,  0.04598753,
         -0.0069641 , -0.05331362],
        [ 0.09086864,  0.09023058,  0.04642185, ...,  0.10861264,
          0.07302871, -0.1115878 ]], dtype=float32), (10000, 128))

now we will try to visualize embedding layer into a great tensorflow tool called projector.
it is a embedding projector.

In [ ]:
import io
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

### MODEL 2: LSTM (LONG SHORT TERM MEMORY)

In [ ]:
# creating model with LSTM layer
from tensorflow.keras import layers
input = layers.Input(shape=(1,), dtype='string')
x = text_vectorizer(input)
x = embedding(x)
x = layers.LSTM(64, return_sequences=True)(x)
x = layers.LSTM(64)(x)
x = layers.Dense(64, activation='relu')(x)
output = layers.Dense(1, activation='sigmoid')(x)
model_2 = tf.keras.Model(input, output, name='model_2')

In [ ]:
model_2.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
lstm (LSTM)                  (None, 15, 64)            49408     
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65  

In [ ]:
train_labels.dtype, val_labels.dtype, train_sentences.dtype, val_sentences.dtype

(dtype('int64'), dtype('int64'), dtype('O'), dtype('O'))

In [ ]:
# compile the model
model_2.compile(loss='binary_crossentropy',
                optimizer = tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [ ]:
# fitting our model_2 on our training data
model_2_history = model_2.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=save_dir, experiment_name='model_2_LSTM')])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20210806-153944
Epoch 1/5
215/215 [==============================] - 9s 26ms/step - loss: 0.2177 - accuracy: 0.9254 - val_loss: 0.5946 - val_accuracy: 0.7835
Epoch 2/5
215/215 [==============================] - 4s 18ms/step - loss: 0.1588 - accuracy: 0.9403 - val_loss: 0.7020 - val_accuracy: 0.7795
Epoch 3/5
215/215 [==============================] - 4s 18ms/step - loss: 0.1374 - accuracy: 0.9467 - val_loss: 0.7487 - val_accuracy: 0.7782
Epoch 4/5
215/215 [==============================] - 4s 18ms/step - loss: 0.1106 - accuracy: 0.9556 - val_loss: 0.9080 - val_accuracy: 0.7756
Epoch 5/5
215/215 [==============================] - 4s 18ms/step - loss: 0.0877 - accuracy: 0.9676 - val_loss: 0.9850 - val_accuracy: 0.7756
